# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

# PART 1:

In [1]:
conda install -y lxml

SyntaxError: invalid syntax (<ipython-input-1-f64f3bd94f4c>, line 1)

In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page=requests.get(wikipedia_link)
wikipedia_page = lh.fromstring(wikipedia_page.content)
tr_elements = wikipedia_page.xpath('//tr')
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = wikipedia_page.xpath('//tr')

#Create empty list
col=[]

i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(name)
    col.append((name,[]))

Postcode
Borough
Neighbourhood



In [5]:
col

[('Postcode', []), ('Borough', []), ('Neighbourhood\n', [])]

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

[len(C) for (title,C) in col]

[288, 288, 288]

In [7]:
# Transforming the dictionary on a dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Cleaning break lines
df = df.replace(r'\n','', regex=True)
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [8]:
#Removing Not assigned zip codes and fixing df index
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.index = range(len(df))
df

,Postcode,Borough,Neighbourhood
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A
5,North York,Lawrence Manor,M6A
6,Etobicoke,Islington Avenue,M9A
7,Scarborough,Rouge,M1B
8,Scarborough,Malvern,M1B
9,North York,Don Mills North,M3B


In [9]:
df.shape

(210, 3)

# PART 2:

In [12]:

!pip install geocoder
import geocoder
import pandas as pd
import numpy as np

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
     |████████████████████████████████| 102kB 17.9MB/s 
     |████████████████████████████████| 81kB 18.1MB/s 


In [13]:
df.head()

,Postcode,Borough,Neighbourhood
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


In [14]:
def get_latlng(post_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(post_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70949500000006, -79.36398897099997]

In [15]:
post_codes = df['Postcode']    
coords = [ get_latlng(post_code) for post_code in post_codes.tolist() ]
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,M3A,43.76826,-79.41263
1,North York,Victoria Village,M4A,43.76826,-79.41263
2,Downtown Toronto,Harbourfront,M5A,43.65952,-79.35543
3,Downtown Toronto,Regent Park,M5A,43.65952,-79.35543
4,North York,Lawrence Heights,M6A,43.76826,-79.41263


In [16]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,M3A,43.768260,-79.412630
1,North York,Victoria Village,M4A,43.768260,-79.412630
2,Downtown Toronto,Harbourfront,M5A,43.659520,-79.355430
3,Downtown Toronto,Regent Park,M5A,43.659520,-79.355430
4,North York,Lawrence Heights,M6A,43.768260,-79.412630
5,North York,Lawrence Manor,M6A,43.768260,-79.412630
6,Etobicoke,Islington Avenue,M9A,43.644400,-79.567140
7,Scarborough,Rouge,M1B,43.772230,-79.256660
8,Scarborough,Malvern,M1B,43.772230,-79.256660
9,North York,Don Mills North,M3B,43.768260,-79.412630


In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

/resources/common/.virtualenv/python2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [20]:
df3 = df[ df.Borough.str.contains('Toronto') ]
toronto_coords = get_latlng('')
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighbourhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto